In [1]:
import os

In [2]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader 
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
#clone the github repo
%pwd

'e:\\Amarnath\\Mastering-GENAI\\18. Mar30th Code Analysis in GEN AI\\Code_Helper_Analysis_GenAI\\research'

In [4]:
!mkdir test_repo

In [5]:
repo_path='test_repo/'
Repo.clone_from("https://github.com/amarnath2706/Code_Helper_Analysis_GenAI",to_path=repo_path)

<git.repo.base.Repo 'e:\\Amarnath\\Mastering-GENAI\\18. Mar30th Code Analysis in GEN AI\\Code_Helper_Analysis_GenAI\\research\\test_repo\\.git'>